<a href="https://colab.research.google.com/github/jarodchristiansen/PythonProjects/blob/master/Binary_Predictors.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install --upgrade -q gspread
!pip install gspread-dataframe
!pip install beautifulsoup4 requests

These requirements should normally be satisfied, they are dependencies for gspread to pull the dataframe

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import statsmodels.api as sm
import seaborn
seaborn.set()
import gspread
from gspread_dataframe import get_as_dataframe, set_with_dataframe
import requests

Import necessary libraries to create dataframe, regression, and formatting of regression (plotting) Gspread creates dataframe from sheets file

In [ ]:
from google.colab import auth
auth.authenticate_user()
from oauth2client.client import GoogleCredentials
gc = gspread.authorize(GoogleCredentials.get_application_default())

This will create authorization through google sheets (follow link that appears and copy authorization code)

In [ ]:
sheet1 = gc.open_by_url('https://docs.google.com/spreadsheets/d/1akfelCo1gEpG6zhpKNT2nGPcuV4aSACv-5fd0zE9PKw/edit#gid=0')

sheet1 is entire csv file URL is sheets file URL

In [ ]:
ws = sheet1.worksheet('Data')

This is the tab in csv file

In [ ]:
data = get_as_dataframe(ws)

titles the dataframe extracted from sheets file

In [ ]:
data

,SAT,Admitted,Gender,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10,Unnamed: 11,Unnamed: 12,Unnamed: 13,Unnamed: 14,Unnamed: 15,Unnamed: 16,Unnamed: 17,Unnamed: 18,Unnamed: 19,Unnamed: 20,Unnamed: 21,Unnamed: 22,Unnamed: 23,Unnamed: 24,Unnamed: 25
0,1363.0,No,Male,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1792.0,Yes,Female,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1954.0,Yes,Female,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1653.0,No,Male,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1593.0,No,Male,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
993,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
994,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
995,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
996,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


This is calling the dataframe and representing from Pandas. NaN cells are empty values

In [ ]:
data.dropna(axis=0, how='all', inplace=True)

dropna() is a methodfrom Pandas to remove NaN cells, axis=0 is rows how is any or all, inplace True removes from the dataframe

In [ ]:
data.dropna(axis=1, how='all', inplace=True)

dropna() is a methodfrom Pandas to remove NaN cells, axis=1 is columns how is any or all, inplace True removes from the dataframe

In [ ]:
data

,SAT,Admitted,Gender
0,1363.0,No,Male
1,1792.0,Yes,Female
2,1954.0,Yes,Female
3,1653.0,No,Male
4,1593.0,No,Male
...,...,...,...
163,1722.0,Yes,Female
164,1750.0,Yes,Male
165,1555.0,No,Male
166,1524.0,No,Male


In [ ]:
data = data.copy()
data['Admitted'] = data['Admitted'].map({'Yes':1, 'No':0})
data['Gender'] = data['Gender'].map({'Female':1, 'Male':0})
data

,SAT,Admitted,Gender
0,1363.0,0,0
1,1792.0,1,1
2,1954.0,1,1
3,1653.0,0,0
4,1593.0,0,0
...,...,...,...
163,1722.0,1,1
164,1750.0,1,0
165,1555.0,0,0
166,1524.0,0,0


In [ ]:
y = data['Admitted']
x1 = data[['SAT','Gender']]

In [ ]:
x = sm.add_constant(x1)
reg_log = sm.Logit(y,x)
results_log = reg_log.fit()
results_log.summary()

Optimization terminated successfully.
         Current function value: 0.120117
         Iterations 10


/usr/local/lib/python3.6/dist-packages/numpy/core/fromnumeric.py:2495: FutureWarning: Method .ptp is deprecated and will be removed in a future version. Use numpy.ptp instead.
  return ptp(axis=axis, out=out, **kwargs)


<class 'statsmodels.iolib.summary.Summary'>
"""
                           Logit Regression Results                           
==============================================================================
Dep. Variable:               Admitted   No. Observations:                  168
Model:                          Logit   Df Residuals:                      165
Method:                           MLE   Df Model:                            2
Date:                Sun, 16 Feb 2020   Pseudo R-squ.:                  0.8249
Time:                        23:07:06   Log-Likelihood:                -20.180
converged:                       True   LL-Null:                       -115.26
Covariance Type:            nonrobust   LLR p-value:                 5.118e-42
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const        -68.3489     16.454     -4.154      0.000    -100.598     -36.100
SAT            0.0406      0.010      4.129      0.000       0.021       0.060
Gender         1.9449      0.846      2.299      0.022       0.287       3.603
==============================================================================

Possibly complete quasi-separation: A fraction 0.27 of observations can be
perfectly predicted. This might indicate that there is complete
quasi-separation. In this case some parameters will not be identified.
"""

The Log-Likelihood is high indicating this regression is effective. 
This is also indicated by the low LLR p-Value



In [ ]:
np.exp(1.94)

6.958750970637272

meaning the odds of a woman getting admitted is 6.95*odds_male more likely. 
or a woman is 6.95% more likely to get admitted with multiple variables. Now we can assess how accurate this model is. 

In [ ]:
np.set_printoptions(formatter={'float' : lambda x: "{0:0.2f}".format(x)})
results_log.predict()

array([0.00, 1.00, 1.00, 0.23, 0.02, 0.99, 1.00, 1.00, 1.00, 0.01, 1.00,
       1.00, 0.76, 0.00, 0.60, 1.00, 0.11, 0.12, 0.51, 1.00, 1.00, 1.00,
       0.00, 0.01, 0.97, 1.00, 0.48, 0.99, 1.00, 0.99, 0.00, 0.83, 0.25,
       1.00, 1.00, 1.00, 0.31, 1.00, 0.23, 0.00, 0.02, 0.45, 1.00, 0.00,
       0.99, 0.00, 0.99, 0.00, 0.00, 0.01, 0.00, 1.00, 0.92, 0.02, 1.00,
       0.00, 0.37, 0.98, 0.12, 1.00, 0.00, 0.78, 1.00, 1.00, 0.98, 0.00,
       0.00, 0.00, 1.00, 0.00, 0.78, 0.12, 0.00, 0.99, 1.00, 1.00, 0.00,
       0.30, 1.00, 1.00, 0.00, 1.00, 1.00, 0.85, 1.00, 1.00, 0.00, 1.00,
       1.00, 0.89, 0.83, 0.00, 0.98, 0.97, 0.00, 1.00, 1.00, 0.03, 0.99,
       0.96, 1.00, 0.00, 1.00, 0.01, 0.01, 1.00, 1.00, 1.00, 0.00, 0.00,
       0.02, 0.33, 0.00, 1.00, 0.09, 0.00, 0.97, 0.00, 0.75, 1.00, 1.00,
       0.01, 0.01, 0.00, 1.00, 0.00, 0.99, 0.57, 0.54, 0.87, 0.83, 0.00,
       1.00, 0.00, 0.00, 0.00, 1.00, 0.04, 0.00, 0.01, 1.00, 0.99, 0.52,
       1.00, 1.00, 0.05, 0.00, 0.00, 0.00, 0.68, 1.

This is creating an array of the predicted probabilities based upon our model. We will compare these with the actual values to see how accurate the model is. 


In [ ]:
np.array(data['Admitted'])

array([0, 1, 1, 0, 0, 1, 1, 1, 1, 0, 1, 1, 1, 0, 0, 1, 0, 0, 1, 1, 1, 1,
       0, 0, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 0, 1, 0, 0, 0, 1, 1, 0,
       1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 1, 0, 1, 1, 1, 1, 0,
       0, 0, 1, 0, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 0, 1, 1, 0, 1,
       1, 1, 0, 0, 1, 1, 0, 1, 1, 0, 1, 1, 1, 0, 1, 0, 0, 1, 1, 1, 0, 0,
       0, 0, 0, 1, 0, 0, 1, 0, 1, 1, 1, 0, 0, 0, 1, 0, 1, 0, 1, 1, 1, 0,
       1, 0, 0, 0, 1, 0, 0, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 1, 1, 1, 1, 1,
       1, 0, 1, 1, 0, 1, 0, 1, 1, 1, 1, 0, 0, 0])

These are the actual values we can compare to the prediction to find the efficiency of our model

In [ ]:
results_log.pred_table()

array([[69.00, 5.00],
       [4.00, 90.00]])

This is creating a matrix of comparative results between the predicted and actual data.

In [ ]:
cm_df = pd.DataFrame(results_log.pred_table())
cm_df.columns = ['Predicted 0', 'Predicted 1']
cm_df = cm_df.rename(index={0:'Actual 0', 1:'Actual 1'})
cm_df

,Predicted 0,Predicted 1
Actual 0,69.0,5.0
Actual 1,4.0,90.0


This is called a confusion matrix. It indicates how 'confused' our model was in predicting our data. In this case it indicates that our model predicted - accurately 69 times, while also predicting 1 accurately 90 times. So in this case we can determine the accuracy of our model. We can take 69+90 = 158 correct predictions of a total of 168 predictions. 94.6%

In [ ]:
cm = np.array(cm_df)
accuracy_train = (cm[0,0]+cm[1,1])/cm.sum()
accuracy_train

0.9464285714285714

Another way of predicting those values wiht code instead of manually processing from the confusion matrix dataframe is to use this piece of code. In this case we can see that our model is 94.6% accuracy. 

In [ ]:
sheet2 = gc.open_by_url('https://docs.google.com/spreadsheets/d/1Ui5QmecF-H2FIbQYSKCXvJX6L7jyEX7y541K4tG4JvA/edit#gid=0')

test = sheet2. worksheet('Data')

test = get_as_dataframe(test)

In [ ]:
test

,SATA1,Admitted,Gender,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10,Unnamed: 11,Unnamed: 12,Unnamed: 13,Unnamed: 14,Unnamed: 15,Unnamed: 16,Unnamed: 17,Unnamed: 18,Unnamed: 19,Unnamed: 20,Unnamed: 21,Unnamed: 22,Unnamed: 23,Unnamed: 24,Unnamed: 25
0,1323.0,No,Male,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1725.0,Yes,Female,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1762.0,Yes,Female,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1777.0,Yes,Male,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1665.0,No,Male,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
993,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
994,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
995,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
996,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
test.dropna(axis=1, how='all', inplace=True)

In [ ]:
test.dropna(axis=0, how='all', inplace=True)

In [ ]:
test

,SATA1,Admitted,Gender
0,1323.0,No,Male
1,1725.0,Yes,Female
2,1762.0,Yes,Female
3,1777.0,Yes,Male
4,1665.0,No,Male
5,1556.0,Yes,Female
6,1731.0,Yes,Female
7,1809.0,Yes,Female
8,1930.0,Yes,Female
9,1708.0,Yes,Male


In [ ]:
test = test.copy()
test['Admitted'] = test['Admitted'].map({'Yes':1, 'No':0})
test['Gender'] = test['Gender'].map({'Female':1, 'Male':0})
test

,SATA1,Admitted,Gender
0,1323.0,0,0
1,1725.0,1,1
2,1762.0,1,1
3,1777.0,1,0
4,1665.0,0,0
5,1556.0,1,1
6,1731.0,1,1
7,1809.0,1,1
8,1930.0,1,1
9,1708.0,1,0


This is to compare our intiail training data compared to our test data. As we can see the columns do not match. We have to correct this to compare the two data sets

In [ ]:
test_actual = test['Admitted']
test_data = test.drop(['Admitted'],axis=1)
test_data = sm.add_constant(test_data)
#test_data = test_data[x.columns.values] if it's ever necessary to reorder the dataframes 
test_data

/usr/local/lib/python3.6/dist-packages/numpy/core/fromnumeric.py:2495: FutureWarning: Method .ptp is deprecated and will be removed in a future version. Use numpy.ptp instead.
  return ptp(axis=axis, out=out, **kwargs)


,const,SATA1,Gender
0,1.0,1323.0,0
1,1.0,1725.0,1
2,1.0,1762.0,1
3,1.0,1777.0,0
4,1.0,1665.0,0
5,1.0,1556.0,1
6,1.0,1731.0,1
7,1.0,1809.0,1
8,1.0,1930.0,1
9,1.0,1708.0,0


In [ ]:
def confusion_matrix(data,actual_values,model):
	
	pred_values = model.predict(data)
	bins=np.array([0,0.5,1])
	cm = np.histogram2d(actual_values, pred_values, bins=bins)[0]
	accuracy = (cm[0,0]+cm[1,1])/cm.sum()
	return cm, accuracy

In [ ]:
cm = confusion_matrix(test_data,test_actual,results_log)
cm

(array([[5.00, 1.00],
        [1.00, 12.00]]), 0.8947368421052632)

This final number of 0.894736 indicates the accuracy of the model. or 89.4% for the test data. If the training data is significantly more accurate than the test model it is an indicator of overfitting.

In [ ]:
cm_df = pd.DataFrame(cm[0])
cm_df.columns = ['Predicted0', 'Predicted 1']
cm_df = cm_df.rename(index={0: 'Actual 0',1: 'Actual 1'})
cm_df

,Predicted0,Predicted 1
Actual 0,5.0,1.0
Actual 1,1.0,12.0


In [ ]:
print ('Misclassifiction rate: '+str((1+1)/19))

Misclassifiction rate: 0.10526315789473684


This is the representation of inaccuracy in the model. known as Misclassification rate